In [ ]:
!pip -q install trl transformers peft bitsandbytes datasets accelerate peft

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, BitsAndBytesConfig
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset, Dataset, DatasetDict
from peft import LoraConfig, get_peft_model
import torch
import json
import time

In [12]:
dataset = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 10884
    })
})

In [13]:
train_test = dataset['train'].train_test_split(test_size=0.2, shuffle=True, seed=42)
test_val = train_test['test'].train_test_split(test_size=200, shuffle=True, seed=42)

dataset = DatasetDict({
    "train": train_test['train'],
    "test": test_val['train'],
    "val": test_val['test']
})

dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 8707
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1977
    })
    val: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 200
    })
})

In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model_name='meta-llama/Llama-3.2-1B-Instruct'